In [3]:
import pandas as pd

In [4]:
# Load Data
df_iter = pd.read_csv('taxi.csv', iterator=True, chunksize=100000, compression='gzip')
df = next(df_iter)
len(df)

100000

In [5]:
df.head().T

,0,1,2,3,4
VendorID,2,1,1,1,2
lpep_pickup_datetime,2019-10-01 00:26:02,2019-10-01 00:18:11,2019-10-01 00:09:31,2019-10-01 00:37:40,2019-10-01 00:08:13
lpep_dropoff_datetime,2019-10-01 00:39:58,2019-10-01 00:22:38,2019-10-01 00:24:47,2019-10-01 00:41:49,2019-10-01 00:17:56
store_and_fwd_flag,N,N,N,N,N
RatecodeID,1,1,1,1,1
PULocationID,112,43,255,181,97
DOLocationID,196,263,228,181,188
passenger_count,1,1,2,1,1
trip_distance,5.88,0.8,7.5,0.9,2.52
fare_amount,18.0,5.0,21.5,5.5,10.0


In [6]:
# Convert to DateTime Format
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])

In [7]:
# Create Postgres Connection
from sqlalchemy import create_engine

user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5432'
db = 'ny_taxi'
table_name = 'green_taxi_trips'

engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')
# Test Connection
engine.connect()

In [8]:
print(pd.io.sql.get_schema(df, name=table_name, con=engine))


CREATE TABLE green_taxi_trips (
	"VendorID" BIGINT, 
	lpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	lpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	store_and_fwd_flag TEXT, 
	"RatecodeID" BIGINT, 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	ehail_fee FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	payment_type BIGINT, 
	trip_type FLOAT(53), 
	congestion_surcharge FLOAT(53)
)




In [9]:
# Adding the column names
df.head(n=0).to_sql(name=table_name, con=engine, if_exists='replace')

0

In [10]:
# Adding first batch of rows
df.to_sql(name=table_name, con=engine, if_exists='append')

1000

In [11]:
# Adding the rest of the rows
from time import time

for df in df_iter:
    t_start = time()
    df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
    df['lpep_dropoff_datetime'] = pd.to_datetime(df['lpep_dropoff_datetime'])
    df.to_sql(name=table_name, con=engine, if_exists='append')
    t_end = time()
    print(f'Inserted another chunk... took {t_end - t_start:.3f} second(s)')
else:
    print('All rows inserted!') 

Inserted another chunk... took 18.877 second(s)
Inserted another chunk... took 18.352 second(s)


/tmp/ipykernel_20215/3749059077.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  for df in df_iter:


Inserted another chunk... took 17.846 second(s)
Inserted another chunk... took 11.338 second(s)
All rows inserted!


In [12]:
# Adding Zone Data
df_zones=pd.read_csv("zones.csv")
table_name = 'zones'
df_zones.head(n=0).to_sql(name=table_name, con=engine, if_exists='replace')
df_zones.to_sql(name=table_name, con=engine, if_exists='append')


265